<a href="https://colab.research.google.com/github/mhoangvslev/bricklink_tool_suites/blob/main/Webrick_Price_Forecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Put your wish list items in this dictionary using key="list name", value="share link"

wishlist = {
    "AK-47": "https://www.webrick.com/wp_collection/share/xajmx14BjJMqhxLfBVcB4Psn2jj1uF1p/",
    "Glock 17": "https://www.webrick.com/wp_collection/share/zGF5c2hvo9rOM8zbnIoydUgIn0BiiVue/",
    "Steyer AUG (Scope &amp; Iron Sight)": "https://www.webrick.com/wp_collection/share/fJ2V62tz0vO7kMx94ifVjoijVN0Y8x55/",
    "UMP-45": "https://www.webrick.com/wp_collection/share/QatSdUiTFPye7NGQo4EytgxPQxIOBhay/",
    "L85A2": "https://www.webrick.com/wp_collection/share/x1MN6B9NOZnvrAWvcKNGQ0LiVTb8KfCY/",
    "Red Dot Sight and Laser Sight": "https://www.webrick.com/wp_collection/share/mqTL21ZoxFKNi67T8bLyWZ8J0mRfcWqX/",
    "MW2 PDSW 528": "https://www.webrick.com/wp_collection/share/J6J3Uhk4VXgZVugB1dhK7sJOjTyvbqSL/",
    "MP5": "https://www.webrick.com/wp_collection/share/LLNNDEuTnmgdTLdYVVVvroUddjk5INSB/"
}

In [ ]:
#@title Define useful functions

import pandas as pd
import re
from bs4 import BeautifulSoup
import requests

def scrape(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'lxml')
    soup

    table = soup.find("form", id="wishlist-view-form").find("table")
    headers = []
    for col in table.find_all("th"):
        headers.append(col.text.strip())

    headers.append("Availability")
    headers[1] = "Item"

    data = pd.DataFrame(columns = headers)
    avail = None
    for table_row in table.find_all("tr")[1:]:

        row = []
        for col in table_row.find_all("td"):
            data_text = col.text.strip()

            qty = col.find("input", class_="input-text qty")
            if qty is not None:
                stock = col.find("span", class_="stock-value")
                if stock is not None:
                    stock = re.sub(r"Stock: (\d+)", r"\1", stock.text)
                else:
                    stock = 0
                qty_value = qty.get("value")
                data_text = f"{qty_value}/{stock}"

            col_attrs = col.attrs.get("class")
            if col_attrs is not None and ("unit-price" in col_attrs or "total-price" in col_attrs):
                data_text = re.sub(r"USD (\d+)", r"\1", data_text)

            row.append(data_text)

        length = len(data)
        if len(row) == 2:
            avail = row[1]
            continue
        elif len(row) != len(headers) - 1:
            continue
        else:
            row.append(avail)
            data.loc[length] = row

    data["Item"] = data["Item"].apply(lambda x: re.sub(r"(.*#\d+)\s+Part ID\s+(\d+)\s+Color ID\s+(\d+)\-(.*)\s+Brick Source\s+(.*)Default\s+Quantity\s+1 piece", r"\1;\2;\3;\4", x).strip())
    data[["Part Name", "PartId", "ColorId", "Color Name"]] = data["Item"].str.split(r";", regex=True, expand=True)
    data[["Quantity", "Stock"]] = data["Qty"].str.split("/", expand=True)
    data.drop(["Item", "Qty"], axis=1, inplace=True)
    return data

In [ ]:
#@title Run this cell every 15 minutes

from tqdm import tqdm
import time

datetime_object = time.time()
timestr = time.strftime("%Y%m%d-%H%M%S")
timestr2 = time.strftime("%m/%d/%y %H:%M:%S")
all_data = []
for set_name, set_url in tqdm(wishlist.items()):
    set_data = scrape(set_url)
    set_data["Project Name"] = set_name
    set_data["Time"] = timestr2
    all_data.append(set_data)

import time
pd.concat(all_data, axis=0).to_csv(f"{timestr}.csv", index=False)

In [ ]:
#@title At the end of the iteration, run this cell once
#@markdown Then upload it to https://giandata-forecast-app-forecastapp-6kanta.streamlit.app/

import glob
pd.concat(glob.glob("*.csv"), axis=0).to_csv(f"webrick_forecast.csv", index=False)